In [12]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import numpy 
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow import keras
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from pylab import rcParams

In [11]:
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [10]:
df = pd.read_csv("APPL.csv", usecols=['open', 'close', 'time'])
df['mid'] = ( df.close + df.open )/ 2

In [13]:
df.head()

,time,open,close,mid
0,2018-08-16T13:30:00Z,52.9500,53.2275,53.08875
1,2018-08-16T14:30:00Z,53.2350,53.3375,53.28625
2,2018-08-16T15:30:00Z,53.3400,53.3375,53.33875
3,2018-08-16T16:30:00Z,53.3375,53.3350,53.33625
4,2018-08-16T17:30:00Z,53.3450,53.3200,53.33250


In [14]:
scaler = MinMaxScaler(feature_range=(0,1))
mid = scaler.fit_transform(df['mid'].to_numpy().reshape(-1,1))

In [15]:
mid.shape

(4083, 1)

In [19]:
train_size = int(len(mid)*.80)
test_size = len(mid) - train_size
train_data,test_data = mid[0:train_size,:],mid[train_size:len(mid),:1]

In [20]:
train_data.shape

(3266, 1)

In [25]:
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
		#dataY.append(a)
	return numpy.array(dataX), numpy.array(dataY)

In [28]:
time_step = 12

X_train,y_train  = create_dataset(train_data, time_step)
X_test,ytest  = create_dataset(test_data, time_step)

X_train.shape

(3253, 12)

In [29]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [30]:
model=Sequential()
#model.add(LSTM(100,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(128,return_sequences=True,activation='relu',input_shape=(12,1)))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(.3))
#model.add(LSTM(50))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam', metrics=['mean_squared_error'])
#model.compile(loss='mean_squared_error',optimizer='adam', metrics=['accuracy'])

model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 12, 128)           66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 12, 128)           131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 12, 128)           131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 12, 100)           91600     
_________________________________________________________________
lstm_4 (LSTM)                (None, 12, 50)            30200     
_________________________________________________________________
dropout (Dropout)            (None, 12, 50)            0         
_________________________________________________________________
flatten (Flatten)            (None, 600)               0

InternalError:  Blas GEMM launch failed : a.shape=(64, 128), b.shape=(128, 512), m=64, n=512, k=128
	 [[{{node sequential/lstm/while/body/_1/sequential/lstm/while/lstm_cell/MatMul_1}}]] [Op:__inference_train_function_11805]

Function call stack:
train_function


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test, ytest),epochs=200,batch_size=64,verbose=1)

### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)